# Azure AI Search עם אינטגרציה של NVIDIA NIM ו-LlamaIndex

במחברת זו, נדגים כיצד לנצל את מודלי הבינה המלאכותית של NVIDIA ואת LlamaIndex כדי ליצור צינור חזק של חיפוש מבוסס-אחזור (RAG). נשתמש במודלים ובתכונות של NVIDIA, נשלב אותם עם Azure AI Search כמאגר וקטורים, ונבצע RAG לשיפור איכות ויעילות החיפוש.

## יתרונות
- **יכולת הרחבה**: ניצול מודלי השפה הגדולים של NVIDIA ו-Azure AI Search לחיפוש יעיל ומתרחב.
- **יעילות כלכלית**: אופטימיזציה של חיפוש ואחזור עם אחסון וקטורי יעיל וטכניקות חיפוש היברידיות.
- **ביצועים גבוהים**: שילוב מודלי שפה חזקים עם חיפוש מבוסס וקטורים לתגובות מהירות ומדויקות יותר.
- **איכות**: שמירה על איכות חיפוש גבוהה באמצעות עיגון תגובות המודלים במסמכים רלוונטיים שנאחזרו.

## דרישות מקדימות
- 🐍 Python 3.9 ומעלה
- 🔗 [שירות Azure AI Search](https://learn.microsoft.com/azure/search/)
- 🔗 מפתח API של NVIDIA לגישה למודלי השפה וההטבעות של NVIDIA דרך שירותי המיקרו של NVIDIA NIM

## תכונות מכוסות
- ✅ אינטגרציה של מודלי שפה של NVIDIA (נשתמש ב-[Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ הטבעות של NVIDIA (נשתמש ב-[nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ מצבי אחזור מתקדמים של Azure AI Search
- ✅ אינדוקס מסמכים עם LlamaIndex
- ✅ RAG באמצעות Azure AI Search ו-LlamaIndex עם מודלי השפה של NVIDIA

בואו נתחיל!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## התקנה ודרישות  
צור סביבת Python באמצעות גרסה >3.10.  

## איך מתחילים!  


כדי להתחיל, אתם צריכים `NVIDIA_API_KEY` כדי להשתמש במודלים של NVIDIA AI Foundation:
1) צרו חשבון חינמי עם [NVIDIA](https://build.nvidia.com/explore/discover).
2) לחצו על המודל שבחרתם.
3) תחת Input, בחרו בלשונית Python, ולחצו **Get API Key** ואז לחצו **Generate Key**.
4) העתיקו ושמרו את המפתח שנוצר כ-NVIDIA_API_KEY. משם, תהיה לכם גישה לנקודות הקצה.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## דוגמה ל-RAG באמצעות LLM ו-Embedding
### 1) אתחול ה-LLM
`llama-index-llms-nvidia`, הידוע גם כ-LLM connector של NVIDIA, מאפשר לך להתחבר וליצור מתוך מודלים תואמים הזמינים בקטלוג ה-API של NVIDIA. ניתן לראות כאן רשימה של מודלים להשלמת שיחה: https://build.nvidia.com/search?term=Text-to-Text

כאן נשתמש ב-**mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) אתחול ההטמעה  
`llama-index-embeddings-nvidia`, הידוע גם כ-מחבר ההטמעות של NVIDIA, מאפשר לך להתחבר וליצור ממודלים תואמים הזמינים בקטלוג ה-API של NVIDIA. בחרנו ב-`nvidia/nv-embedqa-e5-v5` כמודל ההטמעה. ראה כאן רשימה של מודלים להטמעת טקסט: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) צור חנות וקטורים לחיפוש AI של Azure


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

### טען, חלק והעלה מסמכים


In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) יצירת מנוע שאילתות לשאילת שאלות על הנתונים שלך

להלן שאילתה המשתמשת בחיפוש וקטורי טהור ב-Azure AI Search ומבססת את התשובה על מודל השפה שלנו (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

הנה שאילתה באמצעות חיפוש היברידי ב-Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### ניתוח חיפוש וקטורי
תגובת ה-LLM מתארת באופן מדויק את ההשלכות הכלכליות המרכזיות שהוזכרו בטקסט המקור בנוגע לשוק המניות של רוסיה. באופן ספציפי, היא מציינת ששוק המניות הרוסי חווה ירידה משמעותית, ואיבד 40% מערכו, וכי המסחר הופסק בשל המצב המתמשך. תגובה זו תואמת היטב את המידע שסופק במקור, ומעידה על כך שה-LLM זיהה וסיכם נכון את הפרטים הרלוונטיים בנוגע להשפעה על שוק המניות כתוצאה מפעולותיה של רוסיה והסנקציות שהוטלו.

#### הערות על מקורות המידע
מקורות המידע מספקים תיאור מפורט של ההשלכות הכלכליות שרוסיה חוותה בעקבות הסנקציות הבינלאומיות. הטקסט מדגיש ששוק המניות הרוסי איבד 40% מערכו, והמסחר הופסק. בנוסף, הוא מציין השלכות כלכליות נוספות, כמו ירידת ערך הרובל והבידוד הרחב יותר של כלכלת רוסיה. תגובת ה-LLM הצליחה לתמצת את הנקודות המרכזיות ממקורות אלו, תוך התמקדות בהשפעה על שוק המניות כפי שנדרש בשאילתה.


עכשיו, בואו נבחן שאילתה שבה חיפוש היברידי לא מספק תשובה מבוססת היטב:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### חיפוש היברידי: ניתוח תגובת LLM  
תגובת ה-LLM בדוגמת החיפוש ההיברידי מצביעה על כך שההקשר שסופק אינו מציין את התאריך המדויק של פלישת רוסיה לאוקראינה. תגובה זו מרמזת שה-LLM עושה שימוש במידע הזמין במסמכי המקור, אך מכיר בכך שחסרים פרטים מדויקים בטקסט.  

התגובה מדויקת בזיהוי שההקשר מתאר אירועים הקשורים לתוקפנות של רוסיה, אך אינו מציין את תאריך הפלישה הספציפי. הדבר מדגים את יכולתו של ה-LLM להבין את המידע שסופק תוך זיהוי פערים בתוכן. ה-LLM מעודד את המשתמש לפנות למקורות חיצוניים או לרשומות היסטוריות כדי למצוא את התאריך המדויק, ומפגין רמה של זהירות כאשר המידע אינו שלם.  

### ניתוח צמתי מקור  
צמתי המקור בדוגמת החיפוש ההיברידי מכילים קטעים מתוך נאום העוסק בתגובת ארצות הברית לפעולות רוסיה באוקראינה. צמתים אלו מדגישים את ההשפעה הגיאופוליטית הרחבה ואת הצעדים שננקטו על ידי ארצות הברית ובעלות בריתה בתגובה לפלישה, אך אינם מציינים את תאריך הפלישה הספציפי. הדבר תואם את תגובת ה-LLM, שמזהה באופן נכון שההקשר חסר מידע מדויק על התאריך.  


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### ניתוח תגובת LLM: היברידי עם דירוג מחדש
בדוגמה של היברידי עם דירוג מחדש, תגובת ה-LLM מספקת הקשר נוסף על ידי ציון שהאירוע התרחש שישה ימים לפני שניתנה הנאום. הדבר מצביע על כך שה-LLM מסוגל להסיק את תאריך הפלישה בהתבסס על תזמון הנאום, אף על פי שעדיין נדרש לדעת את התאריך המדויק של הנאום לצורך דיוק.

תגובה זו מדגימה יכולת משופרת להשתמש ברמזים מההקשר כדי לספק תשובה אינפורמטיבית יותר. היא מדגישה את היתרון של דירוג מחדש, שבו ה-LLM יכול לגשת למידע רלוונטי יותר ולהעדיפו כדי לספק הערכה קרובה יותר לפרט הרצוי (כלומר, תאריך הפלישה).

### ניתוח צמתי מקור
צמתי המקור בדוגמה זו כוללים התייחסויות לתזמון הפלישה של רוסיה, תוך ציון שהיא התרחשה שישה ימים לפני הנאום. אף על פי שהתאריך המדויק עדיין לא מצוין במפורש, הצמתים מספקים הקשר זמני שמאפשר ל-LLM לתת תגובה מעמיקה יותר. הכללת פרט זה מדגימה כיצד דירוג מחדש יכול לשפר את יכולת ה-LLM לחלץ ולהסיק מידע מתוך ההקשר שסופק, מה שמוביל לתגובה מדויקת ואינפורמטיבית יותר.


**הערה:**  
במחברת זו השתמשנו בשירותי מיקרו של NVIDIA NIM מתוך קטלוג ה-API של NVIDIA.  
ה-API-ים שהוזכרו לעיל, `NVIDIA (llms)`, `NVIDIAEmbedding`, ו-[Azure AI Search Semantic Hybrid Retrieval (עם דירוג מובנה)](https://learn.microsoft.com/azure/search/semantic-search-overview).  
שימו לב, שה-API-ים הללו יכולים גם לתמוך בשירותי מיקרו שמתארחים באופן עצמאי.  

**דוגמה:**  
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")
```



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
